In [1]:
from debugger import EpicDebugger, epic_debugger_decorator
import torch

In [1]:
# simple example where we activate pdb, then can examine self.refs_dict
def broken_fn(x):
    return x / 0

with EpicDebugger(debug_always=False, enabled=True, do_pdb=True, exception_fn=None, normal_debug_fn=None):
    broken_fn(1)

In [8]:
# using the decorator
@epic_debugger_decorator(debug_always=False, enabled=True, do_pdb=True, exception_fn=None, normal_debug_fn=None)
def broken_fn(x):
    return x / 0

broken_fn(1)

In [3]:
# example of passing a function to call on exception, must have refs_dict as the first argument, this will be a dict of references to all collected vars
def another_broken_fn(a_dict, a_list):
    return a_dict + a_list

def print_named_vars(refs_dict, names=None):
    keys = list(refs_dict.keys())
    for name in keys:
        if names is not None:
            if name not in names:
                continue
        print(name, refs_dict[name])

In [4]:
fruits = {"apple": 1, "banana": 2}
numbers = [1, 2, 3]
with EpicDebugger(debug_always=True, do_pdb=False, exception_fn=print_named_vars, names=["fruits", "numbers"]):
    another_broken_fn(fruits, numbers)

********** BEGIN EXCEPTION_FN **********
fruits {'apple': 1, 'banana': 2}
numbers [1, 2, 3]
********** END EXCEPTION_FN **********


Traceback (most recent call last):
  File "/tmp/ipykernel_4170456/1258265313.py", line 4, in <module>
    another_broken_fn(fruits, numbers)
  File "/tmp/ipykernel_4170456/3791443300.py", line 3, in another_broken_fn
    return a_dict + a_list
TypeError: unsupported operand type(s) for +: 'dict' and 'list'


TypeError: unsupported operand type(s) for +: 'dict' and 'list'

In [2]:
thing = {"a": 1, "b": 2, "c": [1, 2, 3], "d": {"e": 1, "f": 2, "g": [(4,5,6), 2, 3]}}
tensor1 = torch.arange(4)
tensor2 = torch.arange(5)

def broken_fn(dictionary, a, b):
    print(a + b)

In [3]:
from functions import print_vars

In [4]:
# a function to print out tensor details and display nested objects
with EpicDebugger(debug_always=True, do_pdb=False, exception_fn=print_vars, names=["thing", "tensor1", "tensor2"]):
    broken_fn(thing, tensor1, tensor2)

Traceback (most recent call last):
  File "/tmp/ipykernel_4170742/514674531.py", line 2, in <module>
    broken_fn(thing, tensor1, tensor2)
  File "/tmp/ipykernel_4170742/3653530737.py", line 6, in broken_fn
    print(a + b)
  File "/home/ubuntu/seq_diffusion/epic-python-debugger/debugger.py", line 57, in __torch_dispatch__
    out = func(*args, **kwargs)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/torch/_ops.py", line 448, in __call__
    return self._op(*args, **kwargs or {})
RuntimeError: The size of tensor a (4) must match the size of tensor b (5) at non-singleton dimension 0


********** BEGIN EXCEPTION_FN **********
thing <class 'dict'>, length:4
- a <class 'int'>
- b <class 'int'>
- c <class 'list'>, length:3
- -  <class 'int'>
- -  <class 'int'>
- -  <class 'int'>
- d <class 'dict'>, length:3
- - e <class 'int'>
- - f <class 'int'>
- - g <class 'list'>, length:3
- - -  <class 'tuple'>, length:3
- - - -  <class 'int'>
- - - -  <class 'int'>
- - - -  <class 'int'>
- - -  <class 'int'>
- - -  <class 'int'>
---------------
tensor1 Device: cpu
tensor1 Type: torch.int64
tensor1 Shape: torch.Size([4])
tensor1 dtype: torch.int64
tensor1 Is Nan: False
tensor1 Is Inf: False
tensor1 Min: 0
tensor1 Max: 3
---------------
tensor2 Device: cpu
tensor2 Type: torch.int64
tensor2 Shape: torch.Size([5])
tensor2 dtype: torch.int64
tensor2 Is Nan: False
tensor2 Is Inf: False
tensor2 Min: 0
tensor2 Max: 4
---------------
********** END EXCEPTION_FN **********


RuntimeError: The size of tensor a (4) must match the size of tensor b (5) at non-singleton dimension 0

In [5]:
# for a function that actually works, but may hide silent errors if tensor values are not checked
tensor1 = torch.arange(4) / 0
tensor2 = torch.arange(4)

def add_tensors(a,b):
    return a + b
    
# runs no problem
out = add_tensors(tensor1, tensor2)

In [12]:
# for this case, because no exception happens, we'll want to use normal_debug_fn and always_debug=True
def throw_if_nan(refs_dict):
    keys = list(refs_dict.keys())
    for name in keys:
        if isinstance(refs_dict[name], torch.Tensor):
            if torch.isnan(refs_dict[name]).any():
                print(f"NaN found in {name}")

In [13]:
@epic_debugger_decorator(debug_always=True, do_pdb=False, normal_debug_fn=throw_if_nan)
def add_tensors(a,b):
    return a + b

out = add_tensors(tensor1, tensor2)

********** BEGIN DEBUG_FN **********
NaN found in a
********** END DEBUG_FN **********
